In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
# Load a pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text input
text = ["This movie was great!", "I did not like the film."]

# Tokenize the text input
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=50)
input_ids = inputs['input_ids']  # Shape: [batch_size, seq_length]

# Convert token IDs to embeddings using a pre-trained model
bert_model = BertModel.from_pretrained('bert-base-uncased')
with torch.no_grad():
    embeddings = bert_model(input_ids).last_hidden_state  # Shape: [batch_size, seq_length, embedding_dim]

In [ ]:
# Define your model class (same as during training)
class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        return out


# Define the LSTM model
embedding_dim = embeddings.size(-1)
hidden_dim = 20  # Example hidden dimension
output_dim = 1   # Binary classification
model = LSTMModel(embedding_dim, hidden_dim, output_dim)

# Load the trained model weights (assuming you have saved them previously)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()  # Set the model to evaluation mode

# Make predictions
with torch.no_grad():  # Disable gradient calculation for inference
    output = model(embeddings)  # Forward pass through the model
    output = output.squeeze(1)  # Remove any extra dimensions if necessary
    prediction = torch.sigmoid(output)  # Apply sigmoid to get probabilities
    predicted_labels = (prediction > 0.5).float()  # Convert probabilities to binary labels

print("Predictions:", predicted_labels)
